# STA 141B Lecture 7

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Link for project repos was posted last week

### Topics

* Web APIs
* The __requests__ package

### Data Sets

* [Star Wars API](https://swapi.co/documentation)
* [iTunes Search API](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/)


### References

* [__requests__ documentation](http://docs.python-requests.org/en/master/)
* Python for Data Analysis, Ch. 6

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/

## Example Questions

1. Which Star Wars movie has the most planets in it? What are the names of the planets in each movie?
2. Can we estimate how many remixes there are of PSY's Gangnam Style?

## Getting Data from the Web

Three ways you can get data from the web, from most to least convenient:

1. Direct download or "data dump"
2. API
3. Scraping

Always look for a direct download first!

## Web APIs

An _application programming interface_ (API) is a collection of functions and data structures for communicating with other software. For instance, whenever you use a Python package, you're using the API created by the package's developers.

Websites sometimes provide an API so that programmers can access content without web scraping.

So how can we call a function in a web API?

### Hypertext Transfer Protocol

The hypertext transfer protocol (HTTP) is a set of rules for communicating over the internet.

For example, your web browser uses HTTP every time you visit a web page. The browser makes a _request_ to the server for the page, and if nothing goes wrong, the server _responds_ with the page. If you have Firefox or Chrome, you can inspect these requests with your browser's web developer tools (`Ctrl-I`).

Several [different kinds of HTTP requests](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods) are possible. Think of these as the different "verbs" you can use when communicating in HTTP.

Many protocols exist for communicating over the internet. For instance, you may have heard of _file transfer protocol_ (FTP) for transferring files, or _simple mail transfer protocol_ (SMTP) for sending/receiving email. However, web APIs almost always use HTTP.

### Representational State Transfer

The most popular kind of web API is a _representational state transfer_ (REST) API. In a REST API:
   
* Each function has a different URL, sometimes called an _endpoint_.
* The server handles separate function calls independently of each other.

We can use the [Star Wars API](https://swapi.co/) to answer our first example question. One of the endpoints in the Star Wars API is `https://swapi.co/api/`. This endpoint returns a list of all other endpoints in the API.

When you first use a web API, check the documentation to find out what the endpoints are and what kind of HTTP requests to use. If the documentation doesn't mention what kind of HTTP request to use, then GET is usually the right choice.

### Making Requests

Python's __requests__ package provides functions for making HTTP requests and is [well-documented](http://docs.python-requests.org/en/master/).

Let's use the endpoint we learned from the Star Wars API.

In [1]:
import requests

In [5]:
response=requests.get("https://swapi.co/api")

A response to an HTTP request always includes a status code that summarizes whether the request was successful. Wikipedia has a full [list of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). Generally,

* 200-299: Your request succeeded.
* 300-399: You need to take further action to complete the request.
* 400-499: Your request wasn't valid (you made a mistake). You've probably seen 404 before!
* 500-599: Your request failed (the server made a mistake).

You can have __requests__ check the status for you with the `.raise_for_status()` method:

Once you have the response, now what? Where's the data?

Different web APIs use different formats. Again, see the documentation. Two common formats are:

* _JavaScript Object Notation_ (JSON): JSON looks and works a lot like Python lists and dictionaries. Lists are surrounded with `[ ]`, and dictionaries are surrounded with `{ }`. There are many Python libraries for reading JSON into lists and dictionaries. Jupyter notebooks are an example of a file in JSON format.

* _eXtensible Markup Language_ (XML): XML uses "tags" denoted by `< >` to mark up sections of text. We'll learn more about XML when we learn about web scraping, since XML is very similar to hypertext markup language (HTML), the language used to build web pages.

The Star Wars API returns data in JSON format.

We can inspect the raw content (bytes) of a response with the `.content` attribute. If we know the response is in a text format, we can use `.text` to see the content as an ordinary Python string.

In [6]:
response.content

b'{"people":"https://swapi.co/api/people/","planets":"https://swapi.co/api/planets/","films":"https://swapi.co/api/films/","species":"https://swapi.co/api/species/","vehicles":"https://swapi.co/api/vehicles/","starships":"https://swapi.co/api/starships/"}'

In [8]:
response.text

'{"people":"https://swapi.co/api/people/","planets":"https://swapi.co/api/planets/","films":"https://swapi.co/api/films/","species":"https://swapi.co/api/species/","vehicles":"https://swapi.co/api/vehicles/","starships":"https://swapi.co/api/starships/"}'

Since the response we got is in JSON format, we'd like to convert the string to lists and dictionaries. The __requests__ package provides a method `.json()` to do this.

In [9]:
response.json()#can grab elements and do stuff with it

{'people': 'https://swapi.co/api/people/',
 'planets': 'https://swapi.co/api/planets/',
 'films': 'https://swapi.co/api/films/',
 'species': 'https://swapi.co/api/species/',
 'vehicles': 'https://swapi.co/api/vehicles/',
 'starships': 'https://swapi.co/api/starships/'}

### Being Polite

Making an HTTP request is not free! It has a real cost in CPU time and also cash. Server administrators will not appreciate it if you make too many requests or make requests too quickly. So:

* Use `time.sleep()` to slow down any requests you make in a loop. Aim for no more than 20-30 requests per second.
* Install and use the __requests_cache__ package to avoid downloading extra data when you make the same request twice.

Failing to be polite can get you banned from websites!

In [20]:
import requests
import requests_cache

ModuleNotFoundError: No module named 'requests_cache'

In [3]:
response=requests.get("https://swapi.com/api")

ConnectionError: HTTPSConnectionPool(host='swapi.com', port=443): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000015B9E3AA9B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
requests_cache.install_cache("my_cache")#do this before scraping

### Answering Example Question 1

In [ ]:
response=requests.get('https://swapi.co/api/films')
response.raise_for_status()
result=response.json()
result

### Query Strings

Most of the functions we use have parameters, and you can pass arguments for those parameters when you call a function.

Endpoints in REST APIs work the same way, but the syntax is different. You can pass arguments by adding `?PARAMETER=ARGUMENT` to the end of the URL. Parameter and argument pairs are separated by `&`. This syntax is called a _query string_.

For instance, Apple provides a web API for the iTunes store, with [documentation](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/). We can use this to try to answer our second example question.

The search endpoint is `https://itunes.apple.com/search`, and the documentation lists several parameters. We can use __requests__ to build the query string automatically.

Every response has a `.url` attribute that shows the URL used for the request.